<a href="https://colab.research.google.com/github/pavelpryadokhin/image-metrics/blob/main/FID%2BIS%2BSSIM%2BPSNR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Сравнение ssim/psnr/mse**

In [ ]:
# @title Загрузка данных

from google.colab import files
import os
import shutil

# Загрузка файла с локального компьютера
uploaded = files.upload()

folder_path = '/content'
# folder_path = '/content/1'


# Укажите путь к загруженному файлу
file_path = list(uploaded.keys())
for file in file_path:
  try:shutil.move(file, folder_path)
  except: pass


In [ ]:
# @title Выбор папки

# folder_path = '/content/check'
folder_path = '/content/3'

file_paths = []  # Список для хранения путей файлов
# Получение списка файлов в папке
files = os.listdir(folder_path)
for file in files:
    if os.path.isfile(os.path.join(folder_path, file)):
        file_paths.append(os.path.join(folder_path, file))


print(file_paths)
file_paths2=file_paths

In [ ]:
import zipfile

# Укажите путь к загруженному архиву для распаковки
zip_path = '/content/Картинки.zip'

# Распаковка архива
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
import os

source_path='/content/images/sources'
original_images_dict = {}
for folder_name in os.listdir(source_path):
    folder_path = os.path.join(source_path, folder_name)
    if os.path.isdir(folder_path):
        image_paths = []
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image_paths.append(image_path)
        original_images_dict[int(folder_name)] = image_paths



images_path = "/content/images/shedevrum"
images_dict = {}
all_images_paths = []
for image_name in os.listdir(images_path):
    image_path = os.path.join(images_path, image_name)
    if os.path.isfile(image_path):
        # key = int(image_name.split(".")[0])
        key = int(image_name.replace(",", ".").split(".")[0])
        if key in images_dict:
            images_dict[key].append(image_path)
        else:
            images_dict[key] = [image_path]
        all_images_paths.append(image_path)
images_dict["all"] = all_images_paths
print(images_dict.keys())
print(original_images_dict.keys())
print(images_dict)

In [ ]:
# @title Функции MSE+SSIM+PSNR

from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv


def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err


def compare_images(imageA, imageB, title,index=1):
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB,channel_axis=index)
    psnr = 20 *np.log10(255) - 10* np.log10(m)
    # figure(title,m, s,psnr,imageA,imageB,index)
    return (m, s,psnr)

def figure(title,m, s,psnr,imageA,imageB,index):
    fig = plt.figure(title)
    plt.suptitle("MSE: %.2f, SSIM: %.2f, PSNR: %.2f" % (m, s,psnr))

    # показать первый рисунок
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(imageA, cmap = plt.cm.gray)
    plt.axis("off")

    # показать второй рисунок
    ax = fig.add_subplot(1, 2, 2)
    if index==2:
      imageB = np.clip(image, 0, 255).astype(np.uint8)  # Приведение к диапазону [0, 255] и типу uint8
    plt.imshow(imageB, cmap = plt.cm.gray)
    plt.axis("off")

    plt.show()




In [ ]:
# @title Вывод MSE+SSIM+PSNR
def msp(path_orinal,file_paths):
  # вывод картинок 1-ч/б  2-цвет
  index=1

  original = cv.imread(path_orinal)
  original = cv.resize(original, (299, 299))

  image_paths=file_paths
  shopped = np.empty((len(image_paths), 299, 299, 3))

  # преобразовать в ч/б
  if index==1:
    original = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
    shopped = np.empty((len(image_paths), 299, 299))

  for i,path in enumerate(image_paths):
      image = cv.imread(path)
      if index==1:
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
      image = cv.resize(image, (299, 299))
      shopped[i] = image

  mse_list = list()
  ssim_list = list()
  psnr_list = list()

  for shop_image in shopped:
    m, s, psnr = compare_images(original, shop_image ,"Original vs. Photoshopped",index)
    mse_list.append(m)
    ssim_list.append(s)
    psnr_list.append(psnr)
    # output(mse_list,ssim_list,psnr_list)
    return mse_list,ssim_list,psnr_list

def output(mse_list,ssim_list,psnr_list):
  mse_mean,mse_std = np.mean(mse_list), np.std(mse_list)
  ssim_mean,ssim_std = np.mean(ssim_list), np.std(ssim_list)
  psnr_mean, psnr_std = np.mean(psnr_list), np.std(psnr_list)
  print("Results:")
  print("-" * 40)
  print("Mean and Standard Deviation:")
  print("-" * 40)
  print(f"MSE:  Mean = {mse_mean:.2f}, Std = {mse_std:.2f}")
  print(f"SSIM: Mean = {ssim_mean:.2f}, Std = {ssim_std:.2f}")
  print(f"PSNR: Mean = {psnr_mean:.2f}, Std = {psnr_std:.2f}")



all_mse= dict()
all_ssim = dict()
all_psnr= dict()
for key in original_images_dict:
  mse_list,ssim_list,psnr_list=[],[],[]
  for path_orinal in original_images_dict[key]:
    res=msp(path_orinal,images_dict[key])
    mse_list.extend(res[0])
    ssim_list.extend(res[1])
    psnr_list.extend(res[2])
  all_mse[key]= [np.mean(mse_list), np.std(mse_list)]
  all_ssim[key] = [np.mean(ssim_list), np.std(ssim_list)]
  all_psnr[key]= [np.mean(psnr_list), np.std(psnr_list)]
print(all_mse)
print()
print(all_ssim)
print()
print(all_psnr)

In [ ]:
# @title Красивый табличный вывод MSE+SSIM+PSNR
print()
print('MSE')
for key, values in sorted(all_mse.items()):
      val1=values[0]
      val2=values[1]
      rounded_val1 = round(val1, 2)
      rounded_val2 = round(val2, 2)
      print(f"{val1:.2f} ({val2:.2f})")
print()
print('SSIM')
for key, values in sorted(all_ssim.items()):
      val1=values[0]
      val2=values[1]
      rounded_val1 = round(val1, 2)
      rounded_val2 = round(val2, 2)
      print(f"{val1:.2f} ({val2:.2f})")
print()
print('PSNR')
for key, values in sorted(all_psnr.items()):
      val1=values[0]
      val2=values[1]
      rounded_val1 = round(val1, 2)
      rounded_val2 = round(val2, 2)
      print(f"{val1:.2f} ({val2:.2f})")

In [ ]:
# @title Рассчет Inception Score
import numpy as np
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
# from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray
from numpy.random import shuffle
import cv2 as cv




def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		new_image = resize(image, new_shape, 0)
		images_list.append(new_image)
	return asarray(images_list)





def calculate_inception_score(images, n_split=1, eps=1E-16):
  model = InceptionV3()
  scores = list()
  # pазделить условные вероятности на группы, управляемые аргументом n_split и установленными по умолчанию, равными 10,
  n_part = floor(images.shape[0] / n_split)
  for i in range(n_split):
  # перебрать условные вероятности в блоках изображений или прогнозов из n_part и вычислить начальную оценку.
    ix_start, ix_end = i * n_part, (i+1) * n_part
    subset = images[ix_start:ix_end]
  # Предварительная обработка изображений для модели InceptionV3 (необходимо для реальных изображений)
    subset = subset.astype('float32')
    subset = scale_images(subset, (299,299,3))
    subset = preprocess_input(subset)
    p_yx = model.predict(subset)
    p_y = expand_dims(p_yx.mean(axis=0), 0)
    kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
    sum_kl_d = kl_d.sum(axis=1)
    avg_kl_d = mean(sum_kl_d)
    is_score = exp(avg_kl_d)
    scores.append(is_score)
  # вычислить и вернуть начальные оценки среднего и стандартного отклонения.
  is_avg, is_std = mean(scores), std(scores)
  print(scores)
  return is_avg, is_std

image_paths=images_dict['all']
images = np.empty((len(image_paths), 299, 299, 3))

for i,path in enumerate(image_paths):
    image = cv.imread(path)
    image = cv.resize(image, (299, 299))
    images[i] = image

shuffle(images)
print('loaded', images.shape)
is_avg, is_std = calculate_inception_score(images)
print('score', round(is_avg, 2),is_avg)



In [ ]:
# @title FID
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from numpy.random import shuffle
import cv2 as cv



def calculate_fid(model, images1, images2,eps=1e-6):
  act1 = model.predict(images1)
  act2 = model.predict(images2)
  mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
  mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
  ssdiff = numpy.sum((mu1 - mu2)**2.0)

  offset = numpy.eye(sigma1.shape[0]) * eps
  covmean = sqrtm((sigma1 + offset).dot(sigma2 + offset))
  if iscomplexobj(covmean):
    covmean = covmean.real
  fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid


def fid(file_paths1,file_paths2,key):
  model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

  image1_paths=file_paths1
  images1 = numpy.empty((len(image1_paths), 299, 299, 3))

  for i,path in enumerate(image1_paths):
      image = cv.imread(path)
      image = cv.resize(image, (299, 299))
      images1[i] = image
  shuffle(images1)

  image2_paths=file_paths2
  images2 = numpy.empty((len(image2_paths), 299, 299, 3))
  for i,path in enumerate(image2_paths):
      image = cv.imread(path)
      image = cv.resize(image, (299, 299))
      images2[i] = image
  shuffle(images2)

  # print('Loaded', images1.shape, images2.shape)
  images1 = images1.astype('float32')
  images2 = images2.astype('float32')

  # pre-process images
  images1 = preprocess_input(images1)
  images2 = preprocess_input(images2)
  # calculate fid
  fid = calculate_fid(model, images1, images2)
  # print(key,':')
  # print('FID: %.3f' % fid)
  return round(fid, 2)

all_fid=dict()
for key in sorted(original_images_dict):
  all_fid[key]=fid(original_images_dict[key],images_dict[key],key)

print(all_fid)
for key,val in sorted(all_fid.items()):
  print(val)



